## Instalando y cargando librerias necesarias

In [ ]:
!pip3 install pandas numpy tabula-py unidecode

In [1]:
# Importar librerias y configurar logging
import pandas as pd
import numpy as np
from pathlib import Path
import tabula
from unidecode import unidecode
import logging
logging.basicConfig(level=logging.DEBUG, filename='log.log',  format='%(asctime)s %(levelname)s %(name)s %(message)s')
logger = logging.getLogger(__name__)


## Pasar carpeta de entrada de fichas en PDF y salida en CSV

In [2]:
# Listar archivos pdf
carpeta_entrada = './pdf'
carpeta_salida = './csv'
lista_pdfs = list(Path(carpeta_entrada).glob('**/*.pdf'))
nombres_pdfs = [pdf.name.replace('.pdf', '') for pdf in lista_pdfs]

## Extraer tablas en crudo

In [3]:
# Extraer tablas en crudo
tablas = []
for pdf in lista_pdfs:
    try:
        tabla = tabula.read_pdf(pdf, pages='all', guess=False, lattice=True)
        tablas.append(tabla)
    except Exception as e:
        tablas.append(None)
        logger.error(f'Error al extraer tablas de {pdf}')
        logger.error(e)
        continue

# Obtener tablas de interes: adjudicado e items
tablas_adjudicado = []
tablas_item = []

# Funciones de ayuda
def remove_header(df):
    return df.T.reset_index().T.reset_index(drop=True)
def get_indexes(list_dfs):
    list_adjudicado, list_item, list_cronograma = [], [], []
    idx_adjudicado, idx_item, idx_cronograma = None, None, None
    for i in range(len(list_dfs)):
        if list_dfs[i].columns.str.contains('Nombre proponente').any():
            list_adjudicado.append(i)
        if list_dfs[i].columns.str.contains('UNSPS').any():
            list_item.append(i)
        if list_dfs[i].columns.str.contains('^Fecha$').any():
            list_cronograma.append(i)
    idx_adjudicado = list_adjudicado[0] if len(list_adjudicado) > 0 else None
    idx_item = list_item[0] if len(list_item) > 0 else None
    idx_cronograma = list_cronograma[0] if len(list_cronograma) > 0 else None
    return idx_adjudicado, idx_item, idx_cronograma

# Recorrer

for idx, tabla in enumerate(tablas):
    try:
        [idx_adjudicado, idx_item, idx_cronograma] = get_indexes(tabla)
        tabla_item = [remove_header(x) for x in tabla[idx_item:idx_cronograma]]
        # tabla_item.columns = tabla_item.columns.str.replace('\r', '', regex=True).str.replace(' ', '').to_series().apply(unidecode)
        for df in tabla_item:
            df.drop(df.columns[-1], axis=1, inplace=True)
            # df.drop(df.index[-1], inplace=True)
            df.replace('\r', ' ', regex=True, inplace=True)
            df.insert(0, 'cuce', nombres_pdfs[idx])
            df.insert(1, 'list_n', idx)
            tablas_item.append(df)
    except Exception as e:
        tablas_item.append(None)
        logger.error(f'Error al procesar tablas de ITEM de la lista: {idx}: {nombres_pdfs[idx]}')
        logger.error(e)
        continue
    try:
        tabla_adjudicado = tabla[idx_adjudicado:idx_item]
        for df in tabla_adjudicado:
            df.insert(0, 'cuce', nombres_pdfs[idx])
            df.insert(1, 'list_n', idx)
            df.replace('\r', ' ', regex=True, inplace=True)
            tablas_adjudicado.append(df)
    except Exception as e:
        tablas_adjudicado.append(None)
        logger.error(f'Error al procesar tablas de ADJUDICADO de la lista: {idx}: {nombres_pdfs[idx]}')
        logger.error(e)
        continue

## Concatenando y limpiando tablas

In [5]:
# Concatenando tablas
tablas_item_cat = pd.concat(tablas_item, ignore_index=True)
tablas_adjudicado_cat = pd.concat(tablas_adjudicado, ignore_index=True)

# Nombrando columnas de items
nombres_cols = ['cuce', 'list_n', 'numero','codigo_unspsc','objeto_de_gasto','descripcion_del_bien_o_servicio','unidad_de_medida','cantidad','precio_referencial_unitario','precio_referencial_total','nombre_o_razon_social_del_proponente_adjudicado','precio_unitario_adjudicado','total_adjudicado','cantidad_recepcionada','precio_unitario_real','monto_real_ejecutado','estado','cronograma']
tablas_item_cat.columns = nombres_cols

# limpiando tablas totales
tablas_totales = tablas_item_cat.query('numero == "Total Adjudicado:" | numero == "Total Referencial:" | numero == "Total Recepcionado:"').iloc[:, 0:8]
tablas_totales.columns = ['cuce', 'list_n', 'tipo_a', 'monto_a', 'tipo_b', 'monto_b', 'tipo_c', 'monto_c']

# limpiando tablas item
tablas_item_clean = tablas_item_cat
tablas_item_clean.query('numero != "#"', inplace=True)
tablas_item_clean.query('numero not in ["Total Adjudicado:", "Total Referencial:", "Total Recepcionado:"]', inplace=True)
tablas_item_clean.query('numero != "Unnamed: 0"', inplace=True)
tablas_item_clean = tablas_item_clean.loc[tablas_item_clean.numero.str.isnumeric() == True]

# limpiando tablas adjudicado
tablas_adjudicado_clean = tablas_adjudicado_cat
tablas_adjudicado_clean.columns = ["cuce", "list_n", "numero","nombre_proponente","monto_adjudicado","nro_de","monto_de","fecha_1","monto","fecha_2","rupe","estado"]
# tablas_adjudicado_clean = tablas_adjudicado_clean.loc[tablas_adjudicado_clean.numero.str.isnumeric() == True]

## Guardar tablas limpias

In [6]:
# Guardando tablas de items limpias
tablas_item_clean.to_csv('csv/tablas_item_clean.csv', index=False)
tablas_adjudicado_clean.to_csv('csv/tablas_adjudicado_clean.csv', index=False)